In [1]:
from qutip import mesolve, basis, jmat
import numpy as np
import scipy.linalg as la
from numpy import angle, pi
from qutip import Qobj, propagator, floquet_modes
import time as time
import matplotlib.pyplot as plt

def floquet_modes_phtr(H, T, args=None, parallel=False, sort=False, U=None):
    if 'opts' in args:
        options = args['opts']
    else:
        options = Options()
        options.rhs_reuse = True
        rhs_clear() 
    if U is None:
        # get the unitary propagator
        U = propagator(H, T, [], args, options=options)

    # find the eigenstates for the propagator
    evals, evecs = la.eig(U.full())

    eargs = angle(evals)

    # make sure that the phase is in the interval [-pi, pi], so that
    # the quasi energy is in the interval [-pi/T, pi/T] where T is the
    # period of the driving.  eargs += (eargs <= -2*pi) * (2*pi) +
    # (eargs > 0) * (-2*pi)
    eargs += (eargs <= -pi) * (2 * pi) + (eargs > pi) * (-2 * pi)
    e_quasi = -eargs / T

    # sort by the quasi energy
    if sort:
        order = np.argsort(-e_quasi)
    else:
        order = list(range(len(evals)))

    # prepare a list of kets for the floquet states
    new_dims = [U.dims[0], [1] * len(U.dims[0])]
    new_shape = [U.shape[0], 1]
    kets_order = [Qobj(np.matrix(evecs[:, o]).T,
                       dims=new_dims, shape=new_shape) for o in order]

    return kets_order, e_quasi[order]

def drive_exact(t, args):
    w = args['omega']
    h = args['h']
    h0 = args['h0']
    return h * np.cos(w*t) + h0

def get_hamiltonians_exact(args):
    N = args['N']
    sx,sy,sz = jmat(N,"x"),jmat(N,"y"),jmat(N,"z")
    kn =  2.0/(N-1)                                      # kacNorm
    H0 = kn * sz **2
    H1 = 2 * sx
    return H0,H1

def floq_evolv_exact(args):
    w = args['omega']
    T = 2 * np.pi/w
    H0, H1 = get_hamiltonians_exact(args)
    H = [H0,[H1,drive_exact]]
    try:
        f_modes_0, f_energies = floquet_modes_phtr(H, T, args=args)
        return f_modes_0, f_energies, w
    except:
        return None

print('definition done!')

definition done!


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from scipy.special import jn_zeros, j0
import math
from qutip import Options
from itertools import compress

#from tqdm import tqdm
import h5py, time
freezing_pts = jn_zeros(0, 500)
start = time.time()

#Ns = [10,20,30,50, 100,150]
#ostp = [1e4,1e5,1e6,1e7,1e11, 1e14]

Ns = [10, 24]
ostp = [1e5, 1e7]
nprocs = 16
# which number of floquet mode we take
nth = 0       
h0 = np.sqrt(3)/2**4
ws = np.exp(np.linspace(np.log(1), np.log(100), 20))

#ws1 = np.exp(np.linspace(np.log(1.0), np.log(5), 10))
#ws2 = np.exp(np.linspace(np.log(6,), np.log(15), 30))
#ws3 = np.exp(np.linspace(np.log(17.), np.log(100.0), 10))
#ws = np.concatenate((ws1, ws3 , ws4))
ws = np.exp(np.linspace(np.log(1), np.log(100), 10))

if __name__ == '__main__':    
    for nn,N in enumerate(Ns):
        start1 = time.time()
        opts = Options(nsteps = 1e4)
        params = [{'h0':h0, 'h': 1/4 * w * freezing_pts[0], 'omega':w, 'N':N, 'opts':opts} for w in ws]
        
        with Pool(processes = nprocs) as p:
            data = p.map(floq_evolv_exact,params)

        evec_f = [np.array(data[j][0])[:,:,0] if data[j] is not None else None for j in range(len(ws))]
        boolfilt = [np.prod(a != None, dtype=bool) for a in evec_f]
        evec_f = np.array(list(compress(evec_f, boolfilt)))
        
        phs_f = [np.array(data[i][1]) if data[i] is not None else None for i in range(len(ws))]
        boolfilt = [np.prod(a != None, dtype=bool) for a in phs_f]
        phs_f = np.array(list(compress(phs_f, boolfilt)))
        
        ws = [data[i][2] if data[i] is not None else None for i in range(len(ws))]
        boolfilt = [np.prod(a != None, dtype=bool) for a in ws]
        ws = np.array(list(compress(ws, boolfilt)))

        evals_single_path = np.zeros(len(ws))
        iprmat_single_path = np.zeros(len(ws))
        iprmat = np.zeros((len(ws),2*N+1))

        sx = jmat(N,"x")
        en, st = sx.eigenstates()

        for xx,w in enumerate(ws):
            #Get the previous Floquet Eigensystem
            if xx == 0:
                evals_prev = phs_f[xx].real
                evecs_prev = evec_f[xx]
            else:
                evals_prev = phs_f[xx-1].real
                evecs_prev = evec_f[xx-1]

            #Sort the Floquet states of this itertaion accordingly to orthonormality with the previous ones
            evals_curr = phs_f[xx]
            evecs_curr = evec_f[xx]
            t = np.copy(evecs_curr)
            condition = np.abs(np.around(evecs_prev.T.conj() @ evecs_curr, 1))
            evals_curr = evals_curr[np.argmax(condition, axis=1)]        
            evecs_curr = evecs_curr[:,np.argmax(condition, axis=1)]

            evals_single_path[xx] = evals_curr[nth]
            f_states = evec_f[xx]
            iprmat[xx,:] = [np.sum([np.abs(Qobj(state).overlap(sx_ev))**4 for sx_ev in st],\
                                   axis=-1) for state in f_states]

            iprmat_single_path[xx] = iprmat[xx,:][nth]

        # save the data
        filename = 'phase_crossover_N' + str(N) +'_.hdf5'

        with h5py.File(filename, 'w') as hf:
            hf.create_dataset('ws', np.shape(ws), data=ws)
            hf.create_dataset('evals_single_path', np.shape(evals_single_path), data=evals_single_path)
            hf.create_dataset('iprmat_single_path', np.shape(iprmat_single_path), data=iprmat_single_path)
            hf.attrs['N'] = N
            hf.attrs['nth'] = nth

        print("For N = ", N, "with", nprocs, "processors : time taken ", (time.time()-start1)/60, ' minutes !')
    print('\n run complete!! \n\n TOTAL TIME TAKEN :', (time.time()-start)/60, ' minutes !')

For N =  10 with 16 processors : time taken  0.019106642405192057  minutes !
For N =  24 with 16 processors : time taken  0.09804427226384481  minutes !

 run complete!! 

 TOTAL TIME TAKEN : 0.1171732227007548  minutes !


In [3]:
!ls phase_crossover_N* > file_phasetran.txt
!cat file_phasetran.txt
!date

phase_crossover_N10_.hdf5
phase_crossover_N120_.hdf5
phase_crossover_N30_.hdf5
Wednesday 29 November 2023 12:17:55 PM IST


In [49]:
import numpy as np

ihlist = [np.random.random((4,4)),np.random.random((4,4)), None]
display(ihlist)
boolfilt = [np.prod(a != None, dtype=bool) for a in ihlist]
np.array(list(compress(ihlist, boolfilt)))

[array([[0.16763775, 0.90080535, 0.79312709, 0.51439138],
        [0.27260725, 0.28000401, 0.45318947, 0.19890146],
        [0.53947264, 0.99296386, 0.92627206, 0.70652632],
        [0.97017074, 0.58974918, 0.11940856, 0.85561888]]),
 array([[0.82687158, 0.92656347, 0.65027796, 0.24252265],
        [0.9112022 , 0.40551863, 0.91539529, 0.47126413],
        [0.57065979, 0.91605396, 0.30176122, 0.1499599 ],
        [0.18446573, 0.78490524, 0.7265288 , 0.77540056]]),
 None]

array([[[0.16763775, 0.90080535, 0.79312709, 0.51439138],
        [0.27260725, 0.28000401, 0.45318947, 0.19890146],
        [0.53947264, 0.99296386, 0.92627206, 0.70652632],
        [0.97017074, 0.58974918, 0.11940856, 0.85561888]],

       [[0.82687158, 0.92656347, 0.65027796, 0.24252265],
        [0.9112022 , 0.40551863, 0.91539529, 0.47126413],
        [0.57065979, 0.91605396, 0.30176122, 0.1499599 ],
        [0.18446573, 0.78490524, 0.7265288 , 0.77540056]]])

In [21]:
ihlist != None

True